In [1]:
import numpy as np
from sklearn import svm

from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from mne.decoding import UnsupervisedSpatialFilter

from src.utils import load_and_regularize_covariance_matrices, calculate_covariance_means, project_to_tangent_and_triu, evaluate_metrics_repeatedly
from src.covariance_means import generalized_eigenvalue_covariance_mean
from pyriemann_qiskit.classification import QuanticSVM, QuanticVQC
from pyriemann.classification import MDM
from pyriemann.channelselection import ElectrodeSelection
from pyriemann.preprocessing import Whitening

%load_ext autoreload
%autoreload 2

[QClass]  Initializing Quantum Classifier


In [12]:
def apply_spatialfiltering(covs, new_n=4):
    n_subject, n_video, n, _ = np.array(covs).shape
    # sf = ElectrodeSelection(nelec=new_n)
    sf = Whitening(metric="logeuclid", dim_red={'n_components': new_n})
    c = np.array(covs).reshape((n_subject * n_video, n, n))
    new_c = sf.fit(c).transform(c).reshape((n_subject, n_video, new_n, new_n))
    return new_c


In [13]:
c_dataset = load_and_regularize_covariance_matrices(deep_features_type='covs_DN20', group_type='CG')
c_dataset_covs = calculate_covariance_means(c_dataset) #Generalized eigenvalue covariance mean
c_dataset_covs = apply_spatialfiltering(c_dataset_covs)
c_dataset_vectors = project_to_tangent_and_triu(c_dataset_covs)

pk_dataset = load_and_regularize_covariance_matrices(deep_features_type='covs_DN20', group_type='PG')
pk_dataset_covs = calculate_covariance_means(pk_dataset)
pk_dataset_covs = apply_spatialfiltering(pk_dataset_covs)
pk_dataset_vectors = project_to_tangent_and_triu(pk_dataset_covs)

Patients:  73%|███████▎  | 8/11 [00:00<00:00, 28.51it/s]

Patients [Tangent space]: 100%|██████████| 11/11 [00:00<00:00, 46.22it/s]


In [14]:
np.array(pk_dataset_vectors).shape

(11, 8, 10)

In [15]:
# Concatenate control and Pk
# pca = PCA(n_components=10)
# reduced_10dim_pkvectors = pca.fit_transform(pk_dataset_vectors)
# reduced_10dim_cvectors = pca.fit_transform(c_dataset_vectors)


dataset_vectors = np.concatenate([c_dataset_vectors, pk_dataset_vectors], axis=0) 
# (22=parkinsons + 22=control, num_videos, num_features = Tangent space flattened mean covariances)
# pca = UnsupervisedSpatialFilter(PCA(8, whiten=True), average=False)
print(dataset_vectors.shape)
n_samples, num_videos, num_features = dataset_vectors.shape

# Reshape the dataset to 2D: (n_samples * num_videos, num_features)
dataset_vectors_reshaped = dataset_vectors.reshape(n_samples * num_videos, num_features)

# Check the shape of the reshaped data
print(dataset_vectors_reshaped.shape)  # Should be (176, 210)


pca = PCA(n_components=5)
reduced10dim = pca.fit_transform(dataset_vectors_reshaped)
res = reduced10dim.reshape(n_samples, num_videos, 5)
print(reduced10dim.shape)
print(res.shape)
# dataset_vectors = np.concatenate([reduced_10dim_cvectors, reduced_10dim_pkvectors], axis=0) # (22=parkinsons + 22=control, num_videos, num_features = Tangent space flattened mean covariances)
dataset_labels = np.concatenate([np.zeros(c_dataset_vectors.shape[0]), np.ones(pk_dataset_vectors.shape[0])])


(22, 8, 10)
(176, 10)
(176, 5)
(22, 8, 5)


In [16]:
# Evaluate metrics
# evaluate_metrics_repeatedly(dataset_vectors = dataset_vectors,
#                       dataset_labels = dataset_labels,                      
#                       estimator = svm.SVC())
evaluate_metrics_repeatedly(dataset_vectors = dataset_vectors,
                      dataset_labels = dataset_labels,                      
                      estimator = QuanticSVM())

[QClass]  Initializing Quantum Classifier


Repetitions:   0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:FidelityQuantumKernel skipped because of time.
                    Using FidelityStatevectorKernel with AerStatevector.
                    Seed cannot be set with FidelityStatevectorKernel.
                    Increase the number of shots to diminish the noise.


[QClass]  Quantum simulation will be performed
GPU optimization disabled. No device found.
[QClass]  Fitting:  (168, 10)
[QClass]  Feature dimension =  10
[QClass]  Quantum backend =  AerSimulator('aer_simulator_statevector')
[QClass]  seed =  853841
[QClass]  SVM initiating algorithm
[QClass]  Training...


                    Using FidelityStatevectorKernel with AerStatevector.
                    Seed cannot be set with FidelityStatevectorKernel.
                    Increase the number of shots to diminish the noise.


[QClass]  Prediction finished.
[QClass]  Quantum simulation will be performed
GPU optimization disabled. No device found.
[QClass]  Fitting:  (168, 10)
[QClass]  Feature dimension =  10
[QClass]  Quantum backend =  AerSimulator('aer_simulator_statevector')
[QClass]  seed =  853841
[QClass]  SVM initiating algorithm
[QClass]  Training...


...

[QClass]  Prediction finished.
[QClass]  Quantum simulation will be performed
GPU optimization disabled. No device found.
[QClass]  Fitting:  (168, 10)
[QClass]  Feature dimension =  10
[QClass]  Quantum backend =  AerSimulator('aer_simulator_statevector')
[QClass]  seed =  853841
[QClass]  SVM initiating algorithm
[QClass]  Training...


                    Using FidelityStatevectorKernel with AerStatevector.
                    Seed cannot be set with FidelityStatevectorKernel.
                    Increase the number of shots to diminish the noise.


[QClass]  Prediction finished.
[QClass]  Quantum simulation will be performed
GPU optimization disabled. No device found.
[QClass]  Fitting:  (168, 10)
[QClass]  Feature dimension =  10
[QClass]  Quantum backend =  AerSimulator('aer_simulator_statevector')
[QClass]  seed =  853841
[QClass]  SVM initiating algorithm
[QClass]  Training...


                    Using FidelityStatevectorKernel with AerStatevector.
                    Seed cannot be set with FidelityStatevectorKernel.
                    Increase the number of shots to diminish the noise.


[QClass]  Prediction finished.
[QClass]  Quantum simulation will be performed
GPU optimization disabled. No device found.
[QClass]  Fitting:  (168, 10)
[QClass]  Feature dimension =  10
[QClass]  Quantum backend =  AerSimulator('aer_simulator_statevector')
[QClass]  seed =  853841
[QClass]  SVM initiating algorithm
[QClass]  Training...
